In [21]:
import numpy as np
import gym
import random

In [2]:
def iniciarQtable(ambiente):
    # Cria uma tabela de zeros com tamanho de acordo com os espaços de observação e ação
    tabelaQ = np.zeros((ambiente.observation_space.n, ambiente.action_space.n))
    return tabelaQ


In [3]:
def politicaEpsilonGreedy(tabelaQ, estado, epsilon):
    # Cria um número aleatório entre 0 e 1
    num_aleatorio = random.uniform(0, 1)
    # Se o número aleatório for maior que epsilon_greedy --> exploitation
    if num_aleatorio > epsilon:
        # Realiza a ação com o valor mais alto de um estado
        action = np.argmax(tabelaQ[estado])
    # else --> exploration
    else:
        # Toma uma ação aleatória
        action = ambiente.action_space.sample()
    return action


In [4]:
def politicaGreedy(tabelaQ, estado):
    # Cria a política Greedy
    # Exploitation: Toma uma ação com o maior valor do estado e action_value
    action = np.argmax(tabelaQ[estado])
    return action


In [5]:
def treinar(episodiosTreino, epsilonMinimo, epsilonMaximo, taxaDecaimento, ambiente, maximoPassos, tabelaQ, taxaAprendizado, gamma):
    for episode in range(episodiosTreino):
        # Reduz epsilon (porque precisamos de cada vez menos exploração)
        epsilon = epsilonMinimo + (epsilonMaximo - epsilonMinimo)*np.exp(-taxaDecaimento*episode)
        # Reseta o ambiente
        estado = ambiente.reset()
        passosTempo = 0
        concluido = False

        # repete
        for passosTempo in range(maximoPassos):
            # Escolhe a ação At usando a política epsilon greedy
            acao = politicaEpsilonGreedy(tabelaQ, estado, epsilon)

            # Toma a ação At e observa Rt+1 e St+1
            # Toma a ação (a) e observa o resultado de estado(s) e reward (r)
            novoEstado, recompensa, concluido, info = ambiente.step(acao)

            # Atualiza Q(s,a) = Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
            tabelaQ[estado][acao] = tabelaQ[estado][acao] = tabelaQ[estado][acao] + taxaAprendizado * \
                (recompensa + gamma *
                 np.max(tabelaQ[novoEstado]) - tabelaQ[estado][acao])

            # Se pronto, termina o episódio
            if concluido:
                break
            # Nosso estado é o novo estado
            estado = novoEstado
    return tabelaQ


In [6]:
def avaliarAgente(ambiente, maximoPassos, episodiosAvaliacao, tabelaQ):
    recompensasTotais = []
    for episodio in range(episodiosAvaliacao):
        # if seed:
        #     estado = ambiente.reset(seed=seed[episodio])
        # else:
        estado = ambiente.reset()
        concluido = False
        recompensasEpisodio = 0
        for passosTempo in range(maximoPassos):
            acao = np.argmax(tabelaQ[estado][:])
            novoEstado, recompensa, concluido, info = ambiente.step(acao)
            recompensasEpisodio += recompensa
            if concluido:
                break
            estado = novoEstado
        recompensasTotais.append(recompensasEpisodio)
    mediaRecompensas = np.mean(recompensasTotais)
    desvioPadrao = np.std(recompensasTotais)
    return mediaRecompensas, desvioPadrao


In [26]:
def verAgente(ambiente, tabelaQ):
    estado = ambiente.reset()
    for passosTempo in range(1000):
        ambiente.render()
        acao = np.argmax(tabelaQ[estado][:])
        novoEstado, recompensa, concluido, info = ambiente.step(acao)
        if concluido:
            break
        estado = novoEstado


In [8]:
# Parâmetros de treino
episodiosTreino = 100000      # Episódios de Treino
taxaAprendizado = 0.7        # Taxa de Aprendizado

# Parâmetros de avaliação
episodiosAvaliacao = 100     # Episódios de Teste

# Parâmetros de Ambiente
ambienteid = "FrozenLake-v1"               # Nome do Ambiente
maximoPassos = 99            # Máximo de passos por episódio
gamma = 0.95                 # Taxa de desconto
sementeAvaliacao = []
# Parâmetros de Exploração
epsilon = 1.0                # Taxa de Exploração
epsilonMaximo = 1.0          # Probabilidade de Exploração no início
epsilonMinimo = 0.05         # Probabilidade de Exploração Mínima
taxaDecaimento = 0.005       # Taxa de Decaimento Exponencial


In [9]:
ambiente = gym.make("FrozenLake-v1", desc=None,
                    map_name="4x4", is_slippery=False)


In [10]:
tabelaQ = iniciarQtable(ambiente)


In [11]:
tabelaQ = treinar(episodiosTreino, epsilonMinimo, epsilonMaximo,
                  taxaDecaimento, ambiente, maximoPassos, tabelaQ, taxaAprendizado, gamma)


In [12]:
mediaAvaliacao, desvioPadrao = avaliarAgente(ambiente, maximoPassos, episodiosAvaliacao, tabelaQ)


In [13]:
print(f"{mediaAvaliacao:.2f}, {desvioPadrao:.2f}")


1.00, 0.00


In [25]:
verAgente(ambiente, tabelaQ)


SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
